In [19]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.11.0             |   py37h89c1867_0         168 KB  conda-forge
    aiohttp-3.7.3              |   py37h5e8e339_2         

hdf5-1.10.6          | 3.1 MB    | ##################################### | 100% 
pyqtwebengine-5.12.1 | 175 KB    | ##################################### | 100% 
libaec-1.0.4         | 31 KB     | ##################################### | 100% 
ld_impl_linux-64-2.3 | 618 KB    | ##################################### | 100% 
libevent-2.1.10      | 1.1 MB    | ##################################### | 100% 
krb5-1.17.2          | 1.4 MB    | ##################################### | 100% 
nbclient-0.5.2       | 60 KB     | ##################################### | 100% 
blosc-1.21.0         | 841 KB    | ##################################### | 100% 
retrying-1.3.3       | 11 KB     | ##################################### | 100% 
fastcache-1.1.0      | 30 KB     | ##################################### | 100% 
cx_oracle-8.1.0      | 193 KB    | ##################################### | 100% 
cloudpickle-1.6.0    | 22 KB     | ##################################### | 100% 
lz4-c-1.9.3          | 179 K

send2trash-1.5.0     | 12 KB     | ##################################### | 100% 
jedi-0.18.0          | 923 KB    | ##################################### | 100% 
pyyaml-5.4.1         | 189 KB    | ##################################### | 100% 
pyzmq-22.0.3         | 526 KB    | ##################################### | 100% 
keras-preprocessing- | 34 KB     | ##################################### | 100% 
gmp-6.2.1            | 806 KB    | ##################################### | 100% 
html5lib-1.1         | 89 KB     | ##################################### | 100% 
ncurses-6.2          | 985 KB    | ##################################### | 100% 
prompt-toolkit-3.0.1 | 244 KB    | ##################################### | 100% 
pexpect-4.8.0        | 47 KB     | ##################################### | 100% 
jupyter_core-4.7.1   | 72 KB     | ##################################### | 100% 
pip-21.0.1           | 1.1 MB    | ##################################### | 100% 
google-pasta-0.2.0   | 42 KB

tensorflow-estimator | 645 KB    | ##################################### | 100% 
gst-plugins-base-1.1 | 2.5 MB    | ##################################### | 100% 
scikit-learn-0.24.1  | 7.5 MB    | ##################################### | 100% 
pyopenssl-20.0.1     | 48 KB     | ##################################### | 100% 
_pytorch_select-0.2  | 2 KB      | ##################################### | 100% 
unixodbc-2.3.9       | 293 KB    | ##################################### | 100% 
python-dateutil-2.8. | 220 KB    | ##################################### | 100% 
numpy-1.20.1         | 5.8 MB    | ##################################### | 100% 
async-timeout-3.0.1  | 11 KB     | ##################################### | 100% 
wheel-0.36.2         | 31 KB     | ##################################### | 100% 
terminado-0.9.2      | 26 KB     | ##################################### | 100% 
zeromq-4.3.4         | 352 KB    | ##################################### | 100% 
jeepney-0.6.0        | 32 KB

done
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.1.0-py_1
  branca             conda-forge/noarch::branca-0.4.2-pyhd8ed1ab_0


In [20]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [21]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [22]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [23]:
neighborhoods_data = newyork_data['features']

In [24]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [29]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [30]:
neighborhoods

AttributeError: 'NoneType' object has no attribute 'items'

Empty DataFrame
Columns: [Borough, Neighborhood, Latitude, Longitude]
Index: []

In [31]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [32]:
neighborhoods.head()

AttributeError: 'NoneType' object has no attribute 'items'

  Borough Neighborhood   Latitude  Longitude
0   Bronx    Wakefield  40.894705 -73.847201
1   Bronx   Co-op City  40.874294 -73.829939
2   Bronx  Eastchester  40.887556 -73.827806
3   Bronx    Fieldston  40.895437 -73.905643
4   Bronx    Riverdale  40.890834 -73.912585

In [33]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [34]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [35]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [36]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

AttributeError: 'NoneType' object has no attribute 'items'

     Borough        Neighborhood   Latitude  Longitude
0  Manhattan         Marble Hill  40.876551 -73.910660
1  Manhattan           Chinatown  40.715618 -73.994279
2  Manhattan  Washington Heights  40.851903 -73.936900
3  Manhattan              Inwood  40.867684 -73.921210
4  Manhattan    Hamilton Heights  40.823604 -73.949688

In [37]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [38]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [71]:
CLIENT_ID = 'HP4W5LPUCQDCN5MDOO1Y3S0XFZM2YEFSGR5ANC5WNTKWALG1' # your Foursquare ID
CLIENT_SECRET = 'LAMVWZ2XXYRMQKABNPZMKF4KS5RP5NAHUIZD2QTMC2ECSMX3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HP4W5LPUCQDCN5MDOO1Y3S0XFZM2YEFSGR5ANC5WNTKWALG1
CLIENT_SECRET:LAMVWZ2XXYRMQKABNPZMKF4KS5RP5NAHUIZD2QTMC2ECSMX3


In [72]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [73]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [74]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radius, LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=HP4W5LPUCQDCN5MDOO1Y3S0XFZM2YEFSGR5ANC5WNTKWALG1&client_secret=LAMVWZ2XXYRMQKABNPZMKF4KS5RP5NAHUIZD2QTMC2ECSMX3&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

In [75]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60371ca843ee31198379cfbb'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 24,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4429abf964a52037f225e3',
       'name': "Arturo's",
       'location': {'address': '5198 Broadway',
        'crossStreet': 'at 225th St.',
        'lat': 40.87441177110231,
        'lng': -73.91027100981574,
        'labeledLatLngs': [{'label'

In [76]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [77]:
from pandas import json_normalize # tranform JSON file into a pandas dataframe

In [78]:
venues = results['response']['groups'][0]['items']

In [79]:
nearby_venues = json_normalize(venues)


In [80]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


In [81]:
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


AttributeError: 'Series' object has no attribute '_mgr'

In [82]:
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

In [83]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

24 venues were returned by Foursquare.


In [86]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [87]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [88]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3175, 7)


AttributeError: 'NoneType' object has no attribute 'items'

  Neighborhood  Neighborhood Latitude  Neighborhood Longitude          Venue  \
0  Marble Hill              40.876551               -73.91066       Arturo's   
1  Marble Hill              40.876551               -73.91066    Bikram Yoga   
2  Marble Hill              40.876551               -73.91066  Tibbett Diner   
3  Marble Hill              40.876551               -73.91066      Starbucks   
4  Marble Hill              40.876551               -73.91066        Dunkin'   

   Venue Latitude  Venue Longitude Venue Category  
0       40.874412       -73.910271    Pizza Place  
1       40.876844       -73.906204    Yoga Studio  
2       40.880404       -73.908937          Diner  
3       40.877531       -73.905582    Coffee Shop  
4       40.877136       -73.906666     Donut Shop  

In [89]:
manhattan_venues.groupby('Neighborhood').count()

AttributeError: 'NoneType' object has no attribute 'items'

                     Neighborhood Latitude  Neighborhood Longitude  Venue  \
Neighborhood                                                                
Battery Park City                       75                      75     75   
Carnegie Hill                           86                      86     86   
Central Harlem                          43                      43     43   
Chelsea                                100                     100    100   
Chinatown                              100                     100    100   
Civic Center                           100                     100    100   
Clinton                                100                     100    100   
East Harlem                             38                      38     38   
East Village                           100                     100    100   
Financial District                     100                     100    100   
Flatiron                               100                     100    100   

In [90]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 330 uniques categories.


In [91]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

AttributeError: 'NoneType' object has no attribute 'items'

  Neighborhood  Accessories Store  Adult Boutique  Afghan Restaurant  \
0  Marble Hill                  0               0                  0   
1  Marble Hill                  0               0                  0   
2  Marble Hill                  0               0                  0   
3  Marble Hill                  0               0                  0   
4  Marble Hill                  0               0                  0   

   African Restaurant  American Restaurant  Antique Shop  Arepa Restaurant  \
0                   0                    0             0                 0   
1                   0                    0             0                 0   
2                   0                    0             0                 0   
3                   0                    0             0                 0   
4                   0                    0             0                 0   

   Argentinian Restaurant  Art Gallery  Art Museum  Arts & Crafts Store  \
0                      

In [92]:
manhattan_onehot.shape

(3175, 331)

In [93]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

AttributeError: 'NoneType' object has no attribute 'items'

           Neighborhood  Accessories Store  Adult Boutique  Afghan Restaurant  \
0     Battery Park City           0.000000        0.000000               0.00   
1         Carnegie Hill           0.000000        0.000000               0.00   
2        Central Harlem           0.000000        0.000000               0.00   
3               Chelsea           0.000000        0.000000               0.00   
4             Chinatown           0.000000        0.000000               0.00   
5          Civic Center           0.000000        0.000000               0.00   
6               Clinton           0.000000        0.000000               0.00   
7           East Harlem           0.000000        0.000000               0.00   
8          East Village           0.000000        0.000000               0.00   
9    Financial District           0.000000        0.000000               0.00   
10             Flatiron           0.000000        0.000000               0.00   
11             Gramercy     

In [94]:
manhattan_grouped.shape

(40, 331)

In [95]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
            venue  freq
0     Coffee Shop  0.08
1           Hotel  0.07
2  Clothing Store  0.05
3            Park  0.05
4             Gym  0.05


----Carnegie Hill----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.05
2  Yoga Studio  0.03
3    Wine Shop  0.03
4  Pizza Place  0.03


----Central Harlem----
                 venue  freq
0   African Restaurant  0.07
1    French Restaurant  0.05
2                  Bar  0.05
3  American Restaurant  0.05
4       Cosmetics Shop  0.05


----Chelsea----
                venue  freq
0         Coffee Shop  0.06
1              Bakery  0.05
2         Art Gallery  0.04
3  Seafood Restaurant  0.03
4               Hotel  0.03


----Chinatown----
                 venue  freq
0               Bakery  0.08
1   Chinese Restaurant  0.07
2         Dessert Shop  0.04
3  American Restaurant  0.04
4    Hotpot Restaurant  0.04


----Civic Center----
                  venue  freq
0           Coffee Shop  0.07
1                 

In [97]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [98]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

AttributeError: 'NoneType' object has no attribute 'items'

        Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0  Battery Park City           Coffee Shop                 Hotel   
1      Carnegie Hill           Coffee Shop                  Café   
2     Central Harlem    African Restaurant     French Restaurant   
3            Chelsea           Coffee Shop                Bakery   
4          Chinatown                Bakery    Chinese Restaurant   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                   Gym                  Park        Clothing Store   
1           Yoga Studio        Cosmetics Shop                   Gym   
2    Chinese Restaurant   American Restaurant        Cosmetics Shop   
3           Art Gallery   American Restaurant    Italian Restaurant   
4          Cocktail Bar          Dessert Shop   American Restaurant   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0         Memorial Site             BBQ Joint          Gourmet Shop   
1           Pizza Plac

In [99]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 0, 0, 0, 1, 1, 3, 0, 1], dtype=int32)

In [100]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

AttributeError: 'NoneType' object has no attribute 'items'

     Borough        Neighborhood   Latitude  Longitude  Cluster Labels  \
0  Manhattan         Marble Hill  40.876551 -73.910660               1   
1  Manhattan           Chinatown  40.715618 -73.994279               0   
2  Manhattan  Washington Heights  40.851903 -73.936900               0   
3  Manhattan              Inwood  40.867684 -73.921210               3   
4  Manhattan    Hamilton Heights  40.823604 -73.949688               4   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0        Sandwich Place        Discount Store                   Gym   
1                Bakery    Chinese Restaurant          Cocktail Bar   
2                  Café                Bakery     Mobile Phone Shop   
3                Lounge    Mexican Restaurant            Restaurant   
4           Pizza Place                  Café           Coffee Shop   

  4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0           Coffee Shop           Pizza Place            

In [101]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [102]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

AttributeError: 'NoneType' object has no attribute 'items'

          Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
1            Chinatown                Bakery    Chinese Restaurant   
2   Washington Heights                  Café                Bakery   
5       Manhattanville    Seafood Restaurant           Coffee Shop   
6       Central Harlem    African Restaurant     French Restaurant   
12     Upper West Side                  Café    Italian Restaurant   
13      Lincoln Square                  Café                 Plaza   
17             Chelsea           Coffee Shop                Bakery   
19        East Village                   Bar           Pizza Place   
20     Lower East Side    Chinese Restaurant           Art Gallery   
21             Tribeca                  Park    Italian Restaurant   
22        Little Italy                  Café    Chinese Restaurant   
25    Manhattan Valley           Coffee Shop           Yoga Studio   

        3rd Most Common Venue 4th Most Common Venue  5th Most Common Venue  \
1          

In [103]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

AttributeError: 'NoneType' object has no attribute 'items'

          Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0          Marble Hill        Sandwich Place        Discount Store   
14             Clinton    Italian Restaurant               Theater   
15             Midtown                 Hotel        Clothing Store   
16         Murray Hill           Coffee Shop        Sandwich Place   
28   Battery Park City           Coffee Shop                 Hotel   
29  Financial District           Coffee Shop           Pizza Place   
32        Civic Center           Coffee Shop  Gym / Fitness Center   
33       Midtown South     Korean Restaurant                 Hotel   
34        Sutton Place           Coffee Shop    Italian Restaurant   
35          Turtle Bay    Italian Restaurant           Coffee Shop   
39        Hudson Yards  Gym / Fitness Center                 Hotel   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                    Gym           Coffee Shop           Pizza Place   
14  Gym / Fitne

In [104]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

AttributeError: 'NoneType' object has no attribute 'items'

       Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
37  Stuyvesant Town                  Park                   Bar   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
37           Coffee Shop  Gym / Fitness Center       Harbor / Marina   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
37          Cocktail Bar              Heliport        Farmers Market   

   9th Most Common Venue 10th Most Common Venue  
37          Skating Rink          Boat or Ferry  

In [105]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

AttributeError: 'NoneType' object has no attribute 'items'

           Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
3                Inwood                Lounge    Mexican Restaurant   
7           East Harlem    Mexican Restaurant         Deli / Bodega   
11     Roosevelt Island                  Park        Scenic Lookout   
26  Morningside Heights                  Park             Bookstore   
36           Tudor City                  Park    Mexican Restaurant   

   3rd Most Common Venue 4th Most Common Venue      5th Most Common Venue  \
3             Restaurant                  Café                   Wine Bar   
7                 Bakery       Thai Restaurant  Latin American Restaurant   
11  Gym / Fitness Center                   Gym      Outdoors & Recreation   
26   American Restaurant           Coffee Shop                       Café   
36                  Café                   Gym              Deli / Bodega   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
3            Pizza Place              

In [106]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

AttributeError: 'NoneType' object has no attribute 'items'

         Neighborhood 1st Most Common Venue    2nd Most Common Venue  \
4    Hamilton Heights           Pizza Place                     Café   
8     Upper East Side           Coffee Shop       Italian Restaurant   
9           Yorkville    Italian Restaurant                      Gym   
10         Lenox Hill    Italian Restaurant             Cocktail Bar   
18  Greenwich Village    Italian Restaurant           Clothing Store   
23               Soho        Clothing Store              Coffee Shop   
24       West Village    Italian Restaurant  New American Restaurant   
27           Gramercy                   Bar              Pizza Place   
30      Carnegie Hill           Coffee Shop                     Café   
31               Noho    Italian Restaurant              Art Gallery   
38           Flatiron                   Spa       Italian Restaurant   

   3rd Most Common Venue    4th Most Common Venue     5th Most Common Venue  \
4            Coffee Shop            Deli / Bodega       